<h1>Algoritmo de jaque-mate</h1>
<p>En este apartado nos vamos a encargar de encontrar si existe una situacion de jaque o jaque-mate en el tablero, recibido como entrada como un array de dimensión 2x2 con valores del 0 al 12, en el que cada numero representa una ficha y el 0 la casilla vacía.</p>


In [1]:
import numpy as np
import copy

In [2]:
not_check_mate_t = np.array([[1,2,3,4,0,3,2,1],
                    [6,6,6,6,6,6,6,6],
                    [0,0,0,0,0,0,0,0],
                    [0,7,0,0,5,0,0,0],
                    [0,0,0,0,0,0,0,0],
                    [0,9,0,0,7,0,8,0],
                    [12,12,12,12,0,12,12,12],
                    [0,8,0,10,11,9,0,0]])

In [3]:
check_mate_t =  np.array([[1,2,3,4,0,3,2,1],
                    [6,6,6,6,6,6,6,6],
                    [0,0,0,0,0,0,0,0],
                    [0,7,0,0,5,0,12,0],
                    [0,0,0,0,0,0,0,0],
                    [0,9,0,0,7,0,8,0],
                    [12,12,12,12,0,12,12,12],
                    [0,8,0,10,11,9,0,0]])

In [4]:
initial_t = np.array([[1,2,3,4,5,3,2,1],
                    [6,6,6,6,6,6,6,6],
                    [0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0],
                    [12,12,12,12,12,12,12,12],
                    [7,8,9,11,10,9,8,7]])

Para empezar, necesitamos una función para cada tipo de ficha (torre,alfil, caballo,...) que, dada la posicion del rey, compruebe si existe jaque por parte de una ficha de dicho tipo.

In [5]:
def valid_pos(pos):
    # Devuelve True cuando la posición está dentro del tablero de 8x8.
    return pos[0]>=0 and pos[1]>=0 and pos[0]<8 and pos[1]<8

In [36]:
def check_rook(t, king, side, k=None): 
    """ 
    Comprueba si hay alguna torre (u otra ficha k) haciendo jaque al rey
    
    Parámetros:
    -----------
    t : array 
        tablero
    king : (int, int)
        posición del rey
    side : str
        color del rey ("white" o "black")
    k : int
        parametro utilizado para calcular jaque de la reina
    """
    if not k:
        k = 7 if side == "white" else 1
    res = False
    for sq in t[king[0]+1:,king[1]]: # down
        if sq != 0:
            if sq == k:
                #print("down")
                res =  True
            
            break

    for sq in np.flip(t[:king[0],king[1]], 0): # up
        if sq != 0:
            if sq == k:
                #print("up")
                res =  True
            
            break

    for sq in t[king[0],king[1]+1:]: # right
        if sq != 0:
            if sq == k:
                #print("right")
                res =  True
            
            break

    for sq in np.flip(t[king[0],:king[1]], 0): # left
        if sq != 0:
            if sq == k:
                #print("left")
                res =  True
            
            break
            
                
    return res

In [7]:
def knight_moves_coor(pos):
    res = []
    for x in [-2,2]:
        for y in [-1,1]:
            if valid_pos((pos[0]+x, pos[1]+y)):
                res.append((pos[0]+x,pos[1]+y))
    for x in [-1,1]:
        for y in [-2,2]:
            if valid_pos((pos[0]+x, pos[1]+y)):
                res.append((pos[0]+x,pos[1]+y))
    return res
            

def check_knight(t, king, side):
    """ 
    Comprueba si hay algun caballo haciendo jaque al rey
    
    Parámetros:
    -----------
    t : array 
        tablero
    king : (int, int)
        posición del rey
    side : str
        color del rey ("white" o "black")
    """
    k = 9 if side == "white" else 3
    res = False
    for i,j in knight_moves_coor(king):
        sq = t[i,j]
        if sq == k:
            res = True
            break
    
    return res

In [8]:
def check_bishop(t, pos, side, k=None):
    """ 
    Comprueba si hay algun alfil (u otra ficha k) haciendo jaque al rey
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int, int)
        posición del rey
    side : str
        color del rey ("white" o "black")
    k : int
        parametro utilizado para calcular jaque de la reina
    """
    if not k:
        k = 8 if side == "white" else 2 # 2 si se buscan blancas y 8 si se buscan negras
    res = False
    a = False
    b = False
    c = False
    d = False
    i = 1
    while(not a or not b or not c or not d):
        if not valid_pos((pos[0]+i, pos[1]+i)):
            a = True
        if not a and t[pos[0]+i,pos[1]+i] != 0:
            if t[pos[0]+i,pos[1]+i] == k:
                res = True
                break
            else:
                a = True
                
        if not valid_pos((pos[0]-i, pos[1]+i)):
            b = True
        if not b and t[pos[0]-i,pos[1]+i] != 0:
            if t[pos[0]-i,pos[1]+i] == k:
                res = True
                break
            else:
                b = True
                
        if not valid_pos((pos[0]-i, pos[1]-i)):
            c = True
        if not c and t[pos[0]-i,pos[1]-i] != 0:
            if t[pos[0]-i,pos[1]-i] == k:
                res = True
                break
            else:
                c = True
                
        if not valid_pos((pos[0]+i, pos[1]-i)):
            d = True
        if not d and t[pos[0]+1,pos[1]-i] != 0:
            if t[pos[0]+1,pos[1]-i] == k:
                res = True
                break
            else:
                d = True
        i += 1
        
    return res
            
            

In [29]:
def check_queen(t, pos, side):
    """ 
    Comprueba si la reina enemiga hace jaque al rey
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int, int)
        posición del rey
    side : str
        color del rey ("white" o "black")
    """
    k = 10 if side == "white" else 4 
    
    return check_rook(t,pos,side,k=k) or check_bishop(t,pos,side,k=k)

In [10]:
def check_king(t, pos, side):
    """ 
    Comprueba si el rey enemigo hace jaque al rey dado
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int, int)
        posición del rey
    side : str
        color del rey ("white" o "black")
    """
    k = 11 if side == "white" else 5
    for x,y in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]:
        if valid_pos((pos[0]+x, pos[1]+y)):
            if t[pos[0]+x, pos[1]+y] != 0:
                if t[pos[0]+x, pos[1]+y] == k:
                    return True
                
    return False

In [11]:
def check_pawn(t, pos, side):
    """ 
    Comprueba si hay algun peon haciendo jaque al rey
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int, int)
        posición del rey
    side : str
        color del rey ("white" o "black")
    """
    k = 12 if side == "white" else 6
    x = 1 if side == "white" else -1
    for y in [-1, 1]:
        if valid_pos((pos[0]+x,pos[1]+y)):
            if t[pos[0]+x,pos[1]+y] == k:
                return True
    return False

Además, necesitamos una función que nos encuentre la posición del rey de cada lado:

In [12]:
def kings(t, side):
    """
    Devuelve la posición del rey del lado dado.
    
    Parámetros:
    -----------
    t : array 
        tablero
    side : str
        color del rey que se busca("white" o "black")
    """
    k = 5 if side == "white" else 11
    pos = tuple(map(lambda x: x[0] if x.size >0 else -1, np.where(t == k)))
        
    if pos[0] < 0 or pos[0] < 0:
        return None
        
    return pos

Con todo esto, podemos comprobar si el rey de un lado está bajo jaque.

In [13]:
def is_check(t, side):
    """
    Devuelve true si el rey del lado ´side´ está bajo jaque.
    
    Parámetros:
    -----------
    t : array 
        tablero
    side : str
        color del rey ("white" o "black")
    """
    
    res = False
    king = kings(t,side)
    #print(king)
    if king:
        if check_rook(t,king, side):
            print("Check ",side," king by rook")
            res = True
        if check_knight(t, king, side):
            print("Check ",side," king by knight")
            res = True
        if check_bishop(t, king, side):
            print("Check ",side," king by bishop")
            res = True
        if check_queen(t, king, side):
            print("Check ",side," king by queen")
            res = True
        if check_king(t, king, side):
            print("Check ",side," king by king")
            res = True
        if check_pawn(t, king, side):
            print("Check ",side," king by pawn")
            res = True
    else:
        res = True
    return res

Ahora, para comprobar si el tablero está en situación de jaque-mate, tenemos que comprobar todos los posibles movimientos del lado que está en jaque.<br>
Para ello, primero nos definimos las funciones que devuelven el movimiento de cada tipo de ficha.

In [14]:
def rook_moves(t, pos):
    """
    Devuelve la lista de posibles tableros resultantes de mover la torre dada en los parámetros.
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int,int)
        posición de la torre
    """
    i,j = pos
    p = t[i,j]
    side = "white" if p < 7 else "black"
    res = []
    wd = [True,True,True,True]
    k = 1
    while(any(wd)):
        for w,(x,y) in enumerate([(i+k, j),(i-k, j),(i, j+k),(i, j-k)]):
            if wd[w] and valid_pos((x,y)) and (t[x,y] == 0 or (side == "white" and t[x,y] > 6) or (side == "black" and t[x,y] < 7)):
                if t[x,y] != 0:
                    wd[w] = False
                s = copy.copy(t)
                s[i,j] = 0
                s[x,y] = p
                res.append(s)
            else:
                wd[w] = False 
                    
        k += 1
        
    return res

In [15]:
def bishop_moves(t, pos):
    """
    Devuelve la lista de posibles tableros resultantes de mover el alfil dado en los parámetros.
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int,int)
        posición del alfil
    """
    i,j = pos
    p = t[i,j]
    side = "white" if p < 7 else "black"
    res = []
    wd = [True,True,True,True]
    k = 1
    while(any(wd)):
        for w,(x,y) in enumerate([(i+k, j+k),(i-k, j+k),(i+k, j-k),(i-k, j-k)]):
            if wd[w] and valid_pos((x,y)) and (t[x,y] == 0 or (side == "white" and t[x,y] > 6) or (side == "black" and t[x,y] < 7)):
                if t[x,y] != 0:
                    wd[w] = False
                s = copy.copy(t)
                s[i,j] = 0
                s[x,y] = p
                res.append(s)
            else:
                wd[w] = False 
                    
        k += 1
        
    return res

In [16]:
def knight_moves(t, pos):
    """
    Devuelve la lista de posibles tableros resultantes de mover el caballo dado en los parámetros.
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int,int)
        posición del caballo
    """
    
    i,j = pos
    p = t[i,j]
    side = "white" if p < 7 else "black"
    res = []
    for x,y in knight_moves_coor(pos):
        if t[x,y] == 0 or (side == "white" and t[x,y] > 6) or (side == "black" and t[x,y] < 7):
            s = copy.copy(t)
            s[i,j] = 0
            s[x,y] = p
            res.append(s)
            
    return res

In [17]:
def queen_moves(t,pos):
    """
    Devuelve la lista de posibles tableros resultantes de mover la reina dada en los parámetros.
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int,int)
        posición de la reina
    """
    res = []
    r_m = rook_moves(t,pos)
    if r_m: 
        res.append(r_m)
        
    b_m = bishop_moves(t,pos)
    if b_m: 
        res.append(b_m)
    return res

In [18]:
def king_moves(t,pos):
    """
    Devuelve la lista de posibles tableros resultantes de mover el rey dado en los parámetros.
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int,int)
        posición del rey
    """
    res = []
    i,j = pos
    p = t[i,j]
    side = "white" if p < 7 else "black"
    for x,y in [(i-1,j-1),(i-1,j),(i-1,j+1),(i,j-1),(i,j+1),(i+1,j-1),(i+1,j),(i+1,j+1)]:
        if valid_pos((x,y)):
            if t[x,y] == 0 or (side == "white" and t[x,y] > 6) or (side == "black" and t[x,y] < 7):
                s = copy.copy(t)
                s[i,j] = 0
                s[x,y] = p
                res.append(s)
    return res
            

In [19]:
def pawn_moves(t, pos):
    """
    Devuelve la lista de posibles tableros resultantes de mover el peón dado en los parámetros.
    
    Parámetros:
    -----------
    t : array 
        tablero
    pos : (int,int)
        posición del peón
    """
    res = []
    i,j = pos
    p = t[i,j]
    side = "white" if p < 7 else "black"
    
    k = 1 if side == "white" else -1
    x = i + k
    
    if t[x, j] == 0:
        s = copy.copy(t)
        s[i,j] = 0
        s[x,j] = p
        res.append(s)

        if ((i == 1 and side == "white") or (i==6 and side == "black")) and t[x+k, j] == 0: # salida del peon
            s = copy.copy(t)
            s[i,j] = 0
            s[x+k,j] = p
            res.append(s)
        
    
    for y in [j-1, j+1]:
        if valid_pos((x,y)) and (t[x,y] != 0) and ((side == "white" and t[x,y] > 6) or (side == "black" and t[x,y] < 7)):
                s = copy.copy(t)
                s[i,j] = 0
                s[x,y] = p
                res.append(s)
    return res

In [20]:
def possible_moves(t, side):
    """
    Devuelve la lista de posibles tableros resultantes de un movimiento del lado dado.
    
    Parámetros:
    -----------
    t : array 
        tablero
    side : str
        lado al que le toca mover
    """
    switcher = {1:rook_moves, 2:bishop_moves, 3: knight_moves, 4: queen_moves, 5: king_moves, 6:pawn_moves}
    res = []
    k = 0 if side == "white" else 6
    for i in range(8):
        for j in range(8):
            sq = t[i,j]
            if sq != 0 and sq > k and sq < k+7:
                moves = switcher[(((sq-1)%6) + 1) ](t,(i,j))
                if moves:
                    res += moves
            
    return res

Ahora, para comprobar si es jaque-mate, solo tenemos que comprobar si existe jaque y en ese caso mirar si hay algun posible movimiento en el que el rey no esté en jaque.

In [21]:
def is_check_mate(t, side):
    """
    Devuelve true si existe jaque-mate al rey del color dado, caso de que no sea jaque mate,
    muestra el posible movimiento en el que el rey no está en jaque.

     Parámetros:
    -----------
    t : array 
        tablero
    side : str
        color del rey que se quiere comprobar
    """
    check = is_check(t, side)
    mate = True
    if check:
        print("check ", side, " king")
        for m in possible_moves(t,side):
            if not is_check(m, side):
                print(m)
                mate = False
                break
            #print(m)

    return check and mate

        

In [22]:
def check_mate(t):
    """
    Muestra por pantalla si existe jaque mate en el tablero dado
    
     Parámetros:
    -----------
    t : array 
        tablero
    """
    for side in ["white", "black"]:
        if is_check_mate(t, side):
            print("check mate", side, "king")
        else:
            print("not check mate for", side, "king")

        

In [34]:
check_mate(initial_t)

not check mate for white king
not check mate for black king


In [35]:
check_mate(check_mate_t)

down
left
Check  white  king by rook
Check  white  king by bishop
check  white  king
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
d

In [25]:
check_mate(not_check_mate_t)

down
left
Check  white  king by rook
Check  white  king by bishop
check  white  king
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king by bishop
down
left
Check  white  king by rook
Check  white  king b